## SERP
https://serpapi.com/dashboard

In [1]:
from crewai import Agent, Task, Crew
from langchain_community.llms import Ollama

c:\Users\asus\anaconda3\envs\crew_ai_agent\Lib\site-packages\pydantic\_internal\_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


<h3> LLM model </h3>

In [2]:
_llama = Ollama(model="mistral", base_url="http://127.0.0.1:11434")

### Tools

In [4]:
from crewai_tools import Tool
import requests
import api_utils
# Function to perform Google Search using SERP API
def google_search(query: str):
    API_KEY = api_utils.SERP_API_KEY # Ensure your SERP_API_KEY is correct
    url = f"https://serpapi.com/search.json?q={query}&api_key={API_KEY}"
    
    try:
        response = requests.get(url, timeout=10)  # Set timeout
        response.raise_for_status()  # Raise error for bad responses (4xx, 5xx)
        data = response.json()
        
        # Extract top 3 results
        results = [
            result.get("snippet", "No description available.")
            for result in data.get("organic_results", [])
        ]
        
        return "\n".join(results[:3]) if results else "No relevant search results found."
    
    except requests.exceptions.RequestException as e:
        return f"Error fetching search results: {str(e)}"

# Define Google Search as a Tool
search_tool = Tool(
    name="Google Search",
    func=google_search,
    description="Quickly search Google for real-time information."
)

### Agent

In [5]:
# Create an Agent with Google Search Tool
query_agent = Agent(
    role="Query Resolver",
    goal="Answer user queries accurately using AI and real-time web search.",
    backstory="An intelligent assistant that leverages AI and web search for precise answers.",
    tools=[search_tool],
    llm=_llama
)

### Task for an agent

In [6]:
    # Define a Single Task
query_task = Task(
    description="Answer the user question: {user_question}. Use AI for general queries and Google Search for real-time information.",
    expected_output="A well-researched, accurate and precise answer based on AI and web search results.",
    agent=query_agent
)

### Creating the crew

In [8]:
# Crew to Manage Execution
crew = Crew(
    agents=[query_agent],
    tasks=[query_task],
    verbose=True
)

# Example Query
# user_question = "What is the weather today in Ratlam, Madhya Pradesh, India?"
# response = crew.kickoff(inputs={"user_question": user_question})

# **Run AI Assistant**
user_question = input("Ask me anything? ")
response = crew.kickoff(inputs={"user_question": user_question})

print("Response:\n", response)

2025-04-03 18:34:35,419 - 6372 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Query Resolver
 [INFO]: == Starting Task: Answer the user question: Veg restaurant in delhi?. Use AI for general queries and Google Search for real-time information.
 

Vegetarian-Friendly Restaurants in New Delhi · 1. K3. 4.8. (2,532). Open now. Indian, Mediterranean$$$$ · 2. Thyme. 4.9. (2,037). Open now.
Triveni Terrace Cafe in Mandi House. Cafe vibes, serves both veg and non-veg but has unique veg options. Burma Burma in multiple locations. Pure ...
Top 10 must try vegetarian restaurants in Delhi. Yes, time for another list! This is specially for my vegetarian friends and is Navratri compliant as well.

 

Vegetarian-Friendly Restaurants in New Delhi · 1. K3. 4.8. (2,532). Open now. Indian, Mediterranean$$$$ · 2. Thyme. 4.9. (2,037). Open now.
Triveni Terrace Cafe in Mandi House. Cafe vibes, serves both veg and non-veg but has unique veg options. Burma Burma in multiple locations. Pure ...
Best vada pav in the world !! If you have eaten anything better, 